In [1]:
import sqlite3
import pandas as pd

df = pd.read_csv('contratos.csv')


In [4]:
df.columns = df.columns.str.strip()

con = sqlite3.connect('ContratosPublicos.db')
cur = con.cursor()

create_tables_sql = """
    CREATE TABLE IF NOT EXISTS Contratos (
        idContrato INTEGER PRIMARY KEY,
        idLocalExecucao INTEGER,
        nifAdjudicatarios INTEGER,
        nifAdjudicante INTEGER,
        idCpv INTEGER,
        tipoContrato TEXT,
        tipoProcedimento TEXT,
        objectoContrato TEXT,
        dataPublicacao DATE,
        dataCelebracaoContrato DATE,
        precoContratual REAL,
        prazoExecucao INTEGER,
        procedimentoCentralizado BOOLEAN,
        descrAcordoQuadro TEXT,
        fundamentacao TEXT,
        FOREIGN KEY (idLocalExecucao) REFERENCES LocaisExecucao(idLocalExecucao),
        FOREIGN KEY (nifAdjudicatarios) REFERENCES Adjudicatarios(nifAdjudicatario),
        FOREIGN KEY (nifAdjudicante) REFERENCES Adjudicantes(nifAdjudicante),
        FOREIGN KEY (idCpv) REFERENCES Cpv(idCpv)
    );

    CREATE TABLE IF NOT EXISTS Adjudicantes (
        nifAdjudicante INTEGER PRIMARY KEY,
        descricaoAdjudicante TEXT
    );

    CREATE TABLE IF NOT EXISTS Adjudicatarios (
        nifAdjudicatario INTEGER PRIMARY KEY,
        descricaoAdjudicatario TEXT
    );

    CREATE TABLE IF NOT EXISTS Cpv (
        idCpv INTEGER PRIMARY KEY,
        descricaoCpv TEXT
    );

    CREATE TABLE IF NOT EXISTS locaisExecucao (
        idLocalExecucao INTEGER PRIMARY KEY,
        codigoMunicipio INT,
        FOREIGN KEY (codigoMunicipio) REFERENCES Municipios(codigoMunicipio)
    );

    CREATE TABLE IF NOT EXISTS Municipios (
        codigoMunicipio INT PRIMARY KEY,
        codigoDistrito INT,
        nomeMunicipio TEXT,
        FOREIGN KEY (codigoDistrito) REFERENCES Distritos(codigoDistrito)
    );

    CREATE TABLE IF NOT EXISTS Distritos (
        codigoDistrito INT PRIMARY KEY,
        codigoPais INT,
        nomeDistrito TEXT,
        FOREIGN KEY (codigoPais) REFERENCES Paises(codigoPais)
    );

    CREATE TABLE IF NOT EXISTS Paises (
        codigoPais INT PRIMARY KEY,
        nomePais TEXT
    );
"""

cur.executescript(create_tables_sql)
con.commit()
con.close()


In [8]:
import pandas as pd
import sqlite3

# Load and process the data
df = pd.read_csv('contratos3.csv')
df.columns = df.columns.str.strip()

adjudicante_id = df['adjudicante'].str.split(' - ').str[0].fillna('Unknown')
adjudicante_name = df['adjudicante'].str.split(' - ').str[1].fillna('Unknown')

adjudicatario_id = df['adjudicatarios'].str.split(' - ').str[0].fillna('Unknown')
adjudicatario_name = df['adjudicatarios'].str.split(' - ').str[1].fillna('Unknown')

idCpv = df['cpv'].str.split(' - ').str[0].fillna('Unknown')
descrCpv = df['cpv'].str.split(' - ').str[1].fillna('Unknown')

idContrato = df['idContrato']
idLocalExecucao = df['idLocalExecucao']
tipoContrato = df['tipoContrato']
tipoProcedimento = df['tipoProcedimento']
objetoContrato = df['objetoContrato']
dataPublicacao = df['dataPublicacao']
dataCelebracaoContrato = df['dataCelebracaoContrato']
precoContratual = df['precoContratual']
prazoExecucao = df['prazoExecucao']
fundamentacao = df['fundamentacao']
procedimentoCentralizado = df['procedimentoCentralizado']
descAcordoQuadro = df['descrAcordoQuadro']

codigoMunicipio = df['codigoMunicipio']
nomeMunicipio = df['Municipio']

codigoDistrito = df['codigoDistrito']
nomeDistrito = df['Distrito']

codigoPais = df['codigoPais']
nomePais = df['Pais']

# Create DataFrames and remove duplicates
adjudicante = pd.DataFrame({
    'nifAdjudicante': adjudicante_id,
    'descricaoAdjucicante': adjudicante_name
}).drop_duplicates()

adjudicatario = pd.DataFrame({
    'nifAdjudicatario': adjudicatario_id,
    'descricaoAdjudicatario': adjudicatario_name
}).drop_duplicates()

cpv = pd.DataFrame({
    'idCpv': idCpv,
    'descricaoCpv': descrCpv
}).drop_duplicates()

LocaisExecucao = pd.DataFrame({
    'idLocalExecucao': idLocalExecucao,
    'codigoMunicipio': codigoMunicipio
}).drop_duplicates()

Municipios = pd.DataFrame({
    'codigoMunicipio': codigoMunicipio,
    'nomeMunicipio': nomeMunicipio,
    'codigoDistrito': codigoDistrito,
}).drop_duplicates()

Distritos = pd.DataFrame({
    'codigoDistrito': codigoDistrito,
    'nomeDistrito': nomeDistrito,
    'codigoPais': codigoPais,
}).drop_duplicates()

Paises = pd.DataFrame({
    'nomePais': nomePais,
    'codigoPais': codigoPais,
}).drop_duplicates()

contratos = pd.DataFrame({
    'idContrato': idContrato,
    'idAdjudicante': adjudicante_id,
    'idAdjudicatario': adjudicatario_id,
    'idLocalizacao': idLocalExecucao,
    'idCpv': idCpv,
    'tipoContrato': tipoContrato,
    'tipoProcedimento': tipoProcedimento,
    'objetoContrato': objetoContrato,
    'dataPublicacao': dataPublicacao,
    'dataCelebracaoContrato': dataCelebracaoContrato,
    'precoContratual': precoContratual,
    'prazoExecucao': prazoExecucao,
    'fundamentacao': fundamentacao,
    'procedimentoCentralizado': procedimentoCentralizado,
    'descAcordoQuadro': descAcordoQuadro,
}).drop_duplicates()

# Save to SQLite database
con = sqlite3.connect('ContratosPublicos.db')

adjudicante.to_sql('Adjudicantes', con, if_exists='replace', index=False)
adjudicatario.to_sql('Adjudicatarios', con, if_exists='replace', index=False)
cpv.to_sql('Cpv', con, if_exists='replace', index=False)
contratos.to_sql('Contratos', con, if_exists='replace', index=False)
LocaisExecucao.to_sql('locaisExecucao', con, if_exists='replace', index=False)
Municipios.to_sql('Municipios', con, if_exists='replace', index=False)
Distritos.to_sql('Distritos', con, if_exists='replace', index=False)
Paises.to_sql('Paises', con, if_exists='replace', index=False)

con.close()


OperationalError: table "locaisExecucao" already exists

In [19]:
import pandas as pd
import sqlite3

# Load the CSV file
df = pd.read_csv('contratos.csv')

# Create a SQLite database connection
con = sqlite3.connect('ContratosPublicos.db')
cur = con.cursor()

# Preprocess and split the data as per the requirements

# Split CPV
df['idCpv'] = df['cpv'].apply(lambda x: x.split(' - ')[0].strip())
df['descricaoCpv'] = df['cpv'].apply(lambda x: x.split(' - ')[1].strip() if ' - ' in x else None)

# Split Adjudicante
df['nifAdjudicante'] = df['adjudicante'].apply(lambda x: x.split(' - ')[0].strip())
df['descricaoAdjudicante'] = df['adjudicante'].apply(lambda x: x.split(' - ')[1].strip() if ' - ' in x else None)

# Split Adjudicatario
df['nifAdjudicatario'] = df['adjudicatarios'].apply(lambda x: x.split(' - ')[0].strip())
df['descricaoAdjudicatario'] = df['adjudicatarios'].apply(lambda x: x.split(' - ')[1].strip() if ' - ' in x else None)

# Split LocalExecucao
df['codigoMunicipio'] = df['localExecucao'].apply(lambda x: x.split(' - ')[0].strip())
df['nomeMunicipio'] = df['localExecucao'].apply(lambda x: x.split(' - ')[1].split(',')[0].strip() if ' - ' in x else None)
df['nomeDistrito'] = df['localExecucao'].apply(lambda x: x.split(',')[-1].strip() if ',' in x else None)
df['codigoPais'] = 1  # Assuming a single country for simplicity
df['nomePais'] = 'Portugal'  # Adjust this as per the data

# Populate Paises
paises = df[['codigoPais', 'nomePais']].drop_duplicates()
paises.to_sql('Paises', con, if_exists='append', index=False)

# Populate Distritos
distritos = df[['nomeDistrito', 'codigoPais']].drop_duplicates()
distritos['codigoDistrito'] = distritos.index + 1  # Generate unique IDs
distritos.to_sql('Distritos', con, if_exists='append', index=False)

# Populate Municipios
municipios = df[['codigoMunicipio', 'nomeMunicipio', 'nomeDistrito']].drop_duplicates()
municipios = municipios.merge(distritos, on='nomeDistrito', how='left')
municipios = municipios[['codigoMunicipio', 'codigoDistrito', 'nomeMunicipio']]
municipios.to_sql('Municipios', con, if_exists='append', index=False)

# Populate LocaisExecucao


# Populate CPV
cpv = df[['idCpv', 'descricaoCpv']].drop_duplicates()
cpv.to_sql('Cpv', con, if_exists='append', index=False)

# Populate Adjudicantes
adjudicantes = df[['nifAdjudicante', 'descricaoAdjudicante']].drop_duplicates()
adjudicantes.to_sql('Adjudicantes', con, if_exists='append', index=False)

# Populate Adjudicatarios
adjudicatarios = df[['nifAdjudicatario', 'descricaoAdjudicatario']].drop_duplicates()
adjudicatarios.to_sql('Adjudicatarios', con, if_exists='append', index=False)

# Populate Contratos
contratos = df[['idcontrato','idCpv', 'tipoContrato', 'tipoProcedimento', 'objetoContrato', 
                'dataPublicacao', 'dataCelebracaoContrato', 'precoContratual', 
                'prazoExecucao', 'procedimentoCentralizado', 'descrAcordoQuadro', 
                'fundamentacao', 'nifAdjudicante', 'nifAdjudicatario']]
contratos = contratos.merge(locais_execucao, on='codigoMunicipio', how='left')
contratos.to_sql('Contratos', con, if_exists='append', index=False)

# Close the database connection
con.close()


AttributeError: 'float' object has no attribute 'split'